<a href="https://colab.research.google.com/github/Jingyi-Z/prc_compare/blob/main/Mapping_gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! rm -rf prc_compare
! git clone https://github.com/Jingyi-Z/prc_compare.git

Cloning into 'prc_compare'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 30 (delta 5), reused 22 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (30/30), 18.00 KiB | 3.00 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [2]:
import numpy as np
from dataclasses import dataclass, field
from typing import Callable, Dict, Tuple
from math import copysign
from scipy.integrate import quad
import pickle
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader,Subset,TensorDataset
from torchvision import datasets, transforms


In [3]:
model_path = {
    'icgs8': 'prc_compare/icgs/icgs8.pkl',
    'icgs4': 'prc_compare/icgs/icgs4_2.pkl',
    'fefet8': 'prc_compare/fefet/fefet8.pkl',
    'fefet4': 'prc_compare/fefet/fefet4.pkl',
    'nores8': 'prc_compare/nores/nores8.pkl',
    'nores4': 'prc_compare/nores/nores4.pkl',
}


In [4]:
def load_mapping(model_name:str) -> Dict[Tuple[int, ...], np.ndarray]:
    mapping_path = model_path[model_name]
    with open(mapping_path, 'rb') as f:
        mapping = pickle.load(f)
    seq_len = len(list(mapping.keys())[0])
    return mapping, seq_len

def imshow_mapping(model_name:str):
  mapping,_ = load_mapping(model_name)
  plt.figure(figsize=(5, 5))
  for index, inp in enumerate(mapping.keys()):
    plt.plot(mapping[inp])
  plt.yscale("log")
  plt.xlabel("Time Step")
  plt.ylabel("Reservoir States")
  plt.title(f"{model_name} reservoir mapping")
  plt.show()

# No reservoir (nores)

## nores8

In [5]:
mapping_nores8,_ = load_mapping("icgs8")

for keys in mapping_nores8.keys():
  mapping_nores8[keys] = list(keys)

with open("nores8.pkl", "wb") as file:
    pickle.dump(mapping_nores8, file)

## nores4


In [6]:
mapping_nores4,_ = load_mapping("fefet4")

for keys in mapping_nores4.keys():
  mapping_nores4[keys] = list(keys)

with open("nores4.pkl", "wb") as file:
    pickle.dump(mapping_nores4, file)

print(mapping_nores4)

{(0, 0, 0, 0): [0, 0, 0, 0], (0, 0, 0, 1): [0, 0, 0, 1], (0, 0, 1, 0): [0, 0, 1, 0], (0, 0, 1, 1): [0, 0, 1, 1], (0, 1, 0, 0): [0, 1, 0, 0], (0, 1, 0, 1): [0, 1, 0, 1], (0, 1, 1, 0): [0, 1, 1, 0], (0, 1, 1, 1): [0, 1, 1, 1], (1, 0, 0, 0): [1, 0, 0, 0], (1, 0, 0, 1): [1, 0, 0, 1], (1, 0, 1, 0): [1, 0, 1, 0], (1, 0, 1, 1): [1, 0, 1, 1], (1, 1, 0, 0): [1, 1, 0, 0], (1, 1, 0, 1): [1, 1, 0, 1], (1, 1, 1, 0): [1, 1, 1, 0], (1, 1, 1, 1): [1, 1, 1, 1]}


# ICGS

## icgs8

In [7]:
mapping_icgs8, _ = load_mapping("icgs8")
print(mapping_icgs8)

{(0, 0, 0, 0, 0, 0, 0, 0): [8.1e-07, 8.1e-07, 8.1e-07, 8.1e-07, 8.1e-07, 8.1e-07, 8.1e-07, 8.1e-07], (0, 0, 0, 0, 0, 0, 0, 1): [8.1e-07, 8.1e-07, 8.1e-07, 8.1e-07, 8.1e-07, 8.1e-07, 8.1e-07, 4.53e-07], (0, 0, 0, 0, 0, 0, 1, 0): [8.1e-07, 8.1e-07, 8.1e-07, 8.1e-07, 8.1e-07, 8.1e-07, 4.53e-07, 6.33e-07], (0, 0, 0, 0, 0, 0, 1, 1): [8.1e-07, 8.1e-07, 8.1e-07, 8.1e-07, 8.1e-07, 8.1e-07, 4.53e-07, 3.09e-07], (0, 0, 0, 0, 0, 1, 0, 0): [8.1e-07, 8.1e-07, 8.1e-07, 8.1e-07, 8.1e-07, 4.53e-07, 6.33e-07, 7.24e-07], (0, 0, 0, 0, 0, 1, 0, 1): [8.1e-07, 8.1e-07, 8.1e-07, 8.1e-07, 8.1e-07, 4.53e-07, 6.33e-07, 3.79e-07], (0, 0, 0, 0, 0, 1, 1, 0): [8.1e-07, 8.1e-07, 8.1e-07, 8.1e-07, 8.1e-07, 4.53e-07, 3.09e-07, 5.48e-07], (0, 0, 0, 0, 0, 1, 1, 1): [8.1e-07, 8.1e-07, 8.1e-07, 8.1e-07, 8.1e-07, 4.53e-07, 3.09e-07, 2.49e-07], (0, 0, 0, 0, 1, 0, 0, 0): [8.1e-07, 8.1e-07, 8.1e-07, 8.1e-07, 4.53e-07, 6.33e-07, 7.24e-07, 7.69e-07], (0, 0, 0, 0, 1, 0, 0, 1): [8.1e-07, 8.1e-07, 8.1e-07, 8.1e-07, 4.53e-07, 6.33e

## icgs4

In [8]:
mapping_icgs4 = {
    (0, 0, 0, 0): [8.1e-07, 8.1e-07, 8.1e-07, 8.1e-07],
    (0, 0, 0, 1): [8.1e-07, 8.1e-07, 8.1e-07, 4.53e-07],
    (0, 0, 1, 0): [8.1e-07, 8.1e-07, 4.53e-07, 7.77e-07],
    (0, 0, 1, 1): [8.1e-07, 8.1e-07, 4.53e-07, 4.23e-07],
    (0, 1, 0, 0): [8.1e-07, 4.53e-07, 7.77e-07, 8.08e-07],
    (0, 1, 0, 1): [8.1e-07, 4.53e-07, 7.77e-07, 4.5e-07],
    (0, 1, 1, 0): [8.1e-07, 4.53e-07, 4.23e-07, 7.74e-07],
    (0, 1, 1, 1): [8.1e-07, 4.53e-07, 4.23e-07, 4.21e-07],
    (1, 0, 0, 0): [4.53e-07, 7.77e-07, 8.08e-07, 8.1e-07],
    (1, 0, 0, 1): [4.53e-07, 7.77e-07, 8.08e-07, 4.53e-07],
    (1, 0, 1, 0): [4.53e-07, 7.77e-07, 4.5e-07, 7.77e-07],
    (1, 0, 1, 1): [4.53e-07, 7.77e-07, 4.5e-07, 4.23e-07,],
    (1, 1, 0, 0): [4.53e-07, 4.23e-07, 7.74e-07, 8.07e-07],
    (1, 1, 0, 1): [4.53e-07, 4.23e-07, 7.74e-07, 4.5e-07],
    (1, 1, 1, 0): [4.53e-07, 4.23e-07, 4.21e-07, 7.74e-07],
    (1, 1, 1, 1): [4.53e-07, 4.23e-07, 4.21e-07, 4.2e-07]
    }


with open("icgs4.pkl", "wb") as file:
    pickle.dump(mapping_icgs4, file)
